In [1]:
%run ../../Utils/yp_utils.py

# Initial setup

In [2]:
paper_pmid = 15087496
paper_name = 'thorpe_dawes_2004' 

In [3]:
datasets = pd.read_csv('extras/YeastPhenome_' + str(paper_pmid) + '_datasets_list.txt', sep='\t', header=None, names=['dataset_id', 'name'])

In [4]:
datasets.set_index('dataset_id', inplace=True)

# Load & process the data

In [5]:
original_data = pd.read_excel('raw_data/05888Table2.xlsx', sheet_name='Total')

In [6]:
print('Original data dimensions: %d x %d' % (original_data.shape))

Original data dimensions: 685 x 15


In [7]:
original_data.head()

,Gene,ORF,Gene function,CHP,Unnamed: 4,Diamide,Unnamed: 6,H2O2,Unnamed: 8,LoaOOH,Unnamed: 10,Menadione,Unnamed: 12,Glycerol,Slow growth
0,AAD6,YFL056C,Predicted aryl alcohol dehydrogenase N-termina...,NaN,NaN,S,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Slow Growth
1,ABF2,YMR072W,Abundant mitochondrial DNA-binding protein req...,NaN,NaN,R,NaN,S,5.0,NaN,NaN,NaN,NaN,no growth,NaN
2,ACO1,YLR304C,Aconitate hydratase (aconitase); converts citr...,NaN,NaN,S,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,ADA2,YDR448W,Component of two nucleosomal histone acetyltra...,S,6.0,S,3.0,S,NaN,S,NaN,NaN,NaN,no growth,Slow Growth
4,"ADE5,7",YGL234W,Phosphoribosylamine-glycine ligase (GARSase) p...,NaN,NaN,S,2.0,S,NaN,S,NaN,NaN,NaN,NaN,NaN


In [8]:
original_data['orf'] = original_data['ORF'].astype(str)

In [9]:
# Eliminate all white spaces & capitalize
original_data['orf'] = clean_orf(original_data['orf'])

In [10]:
# Translate to ORFs 
original_data['orf'] = translate_sc(original_data['orf'], to='orf')

In [11]:
# Make sure everything translated ok
t = looks_like_orf(original_data['orf'])
print(original_data.loc[~t,])

Empty DataFrame
Columns: [Gene, ORF, Gene function, CHP, Unnamed: 4, Diamide, Unnamed: 6, H2O2, Unnamed: 8, LoaOOH, Unnamed: 10, Menadione, Unnamed: 12, Glycerol, Slow growth, orf]
Index: []


In [12]:
original_data.set_index('orf', inplace=True)

In [13]:
original_data1 = original_data.loc[:,['CHP','Diamide','H2O2','LoaOOH','Menadione']].copy()
original_data2 = original_data.loc[:,['Unnamed: 4','Unnamed: 6','Unnamed: 8','Unnamed: 10','Unnamed: 12']].copy()

In [14]:
data_switch = {'S': -1, 'R': 1, 'nan': 0, 'NAN': 0}
for c in original_data1.columns:
    original_data1[c] = original_data1[c].apply(lambda x: str(x).strip().upper())
    original_data1[c] = original_data1[c].apply(lambda x: data_switch[x] if x in data_switch.keys() else np.nan)

In [15]:
original_data2 = original_data2.apply(pd.to_numeric, axis=1, errors='coerce')
original_data2 = original_data2 + 1
original_data2[original_data2.isnull()] = 1

In [16]:
original_data = pd.DataFrame(index=original_data1.index, columns=original_data1.columns, data=np.multiply(original_data1.values, original_data2.values) )

In [17]:
original_data.head()

,CHP,Diamide,H2O2,LoaOOH,Menadione
orf,,,,,
YFL056C,0.0,-4.0,0.0,0.0,0.0
YMR072W,0.0,1.0,-6.0,0.0,0.0
YLR304C,0.0,-1.0,0.0,0.0,0.0
YDR448W,-7.0,-4.0,-1.0,-1.0,0.0
YGL234W,0.0,-3.0,-1.0,-1.0,0.0


In [18]:
original_data = original_data.groupby(original_data.index).mean()

In [19]:
original_data.shape

(678, 5)

# Prepare the final dataset

In [20]:
data = original_data.copy()

In [21]:
dataset_ids = [4959, 4957, 488, 4954, 4953]
datasets = datasets.reindex(index=dataset_ids)

In [22]:
lst = [datasets.index.values, ['value']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data.columns = idx

In [23]:
data.head()

dataset_id,4959,4957,488,4954,4953
data_type,value,value,value,value,value
orf,,,,,
YAL002W,0.0,-3.0,-1.0,0.0,0.0
YAL009W,0.0,-3.0,-1.0,-1.0,0.0
YAL011W,0.0,0.0,0.0,0.0,-1.0
YAL013W,0.0,-3.0,0.0,-1.0,0.0
YAL021C,0.0,-3.0,0.0,0.0,0.0


## Subset to the genes currently in SGD

In [24]:
genes = pd.read_csv(path_to_genes, sep='\t', index_col='id')
genes = genes.reset_index().set_index('systematic_name')
gene_ids = genes.reindex(index=data.index.values)['id'].values
num_missing = np.sum(np.isnan(gene_ids))
print('ORFs missing from SGD: %d' % num_missing)

ORFs missing from SGD: 1


In [25]:
data['gene_id'] = gene_ids
data = data.loc[data['gene_id'].notnull()]
data['gene_id'] = data['gene_id'].astype(int)
data = data.reset_index().set_index(['gene_id','orf'])

data.head()

,dataset_id,4959,4957,488,4954,4953
,data_type,value,value,value,value,value
gene_id,orf,,,,,
2,YAL002W,0.0,-3.0,-1.0,0.0,0.0
7,YAL009W,0.0,-3.0,-1.0,-1.0,0.0
9,YAL011W,0.0,0.0,0.0,0.0,-1.0
11,YAL013W,0.0,-3.0,0.0,-1.0,0.0
19,YAL021C,0.0,-3.0,0.0,0.0,0.0


# Normalize

In [26]:
data_norm = normalize_phenotypic_scores(data, has_tested=False)

In [27]:
# Assign proper column names
lst = [datasets.index.values, ['valuez']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data_norm.columns = idx

In [28]:
data_norm[data.isnull()] = np.nan
data_all = data.join(data_norm)

data_all.head()

,dataset_id,4959,4957,488,4954,4953,4959,4957,488,4954,4953
,data_type,value,value,value,value,value,valuez,valuez,valuez,valuez,valuez
gene_id,orf,,,,,,,,,,
2,YAL002W,0.0,-3.0,-1.0,0.0,0.0,0.0,-3.328420,-1.021308,0.000000,0.000000
7,YAL009W,0.0,-3.0,-1.0,-1.0,0.0,0.0,-3.328420,-1.021308,-1.141733,0.000000
9,YAL011W,0.0,0.0,0.0,0.0,-1.0,0.0,0.011245,-0.030038,0.000000,-2.184786
11,YAL013W,0.0,-3.0,0.0,-1.0,0.0,0.0,-3.328420,-0.030038,-1.141733,0.000000
19,YAL021C,0.0,-3.0,0.0,0.0,0.0,0.0,-3.328420,-0.030038,0.000000,0.000000


# Print out

In [29]:
for f in ['value','valuez']:
    df = data_all.xs(f, level='data_type', axis=1).copy()
    df.columns = datasets['name'].values
    df = df.droplevel('gene_id', axis=0)
    df.to_csv(paper_name + '_' + f + '.txt', sep='\t')

# Save to DB

In [30]:
from IO.save_data_to_db3 import *

In [31]:
save_data_to_db(data_all, paper_pmid)

  0%|          | 0/5 [00:00<?, ?it/s]

Deleting all datasets for PMID 15087496...
Inserting the new data...


100%|██████████| 5/5 [00:05<00:00,  1.01s/it]

Updating the data_modified_on field...
